In [1]:
from typing import Dict
from l5kit.data.map_api import MapAPI
# from l5kit.environment.envs.l5_env import GymStepOutput, SimulationConfigGym
from l5kit.environment.envs.l5_env2 import GymStepOutput, SimulationConfigGym, L5Env2
from l5kit.environment.envs.l5_env import GymStepOutput, SimulationConfigGym, L5Env
# Dataset is assumed to be on the folder specified
# in the L5KIT_DATA_FOLDER environment variable
import gym
import ray
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune.logger import pretty_print
import ray.rllib.algorithms.ppo as ppo
from ray import tune
from ray.rllib.models import ModelCatalog
# from wrapper import L5EnvWrapper
# from src.customEnv.wr import wrapper
# from customModel.customModel import TorchGNCNN, TorchAttentionModel, TorchAttentionModel2
import os

from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from tempfile import gettempdir

from l5kit.configs import load_config_data
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDatasetVectorized
from l5kit.planning.vectorized.closed_loop_model import VectorizedUnrollModel
from l5kit.planning.vectorized.open_loop_model import VectorizedModel
from l5kit.vectorization.vectorizer_builder import build_vectorizer

In [2]:
# get environment config
os.environ["L5KIT_DATA_FOLDER"] = "/workspace/datasets"
# env_config_path = '/workspace/source/src/configs/gym_vectorizer_config.yaml'
env_config_path = '/workspace/source/src/configs/gym_rasterizer_config.yaml'
dmg = LocalDataManager(None)
cfg = load_config_data(env_config_path)
###############

In [3]:
# model_path = "/home/pronton/rl/l5kit/examples/urban_driver/OL_HS.pt"
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = torch.load(model_path).to(device)
# # model = SAC.load("/home/pronton/rl/l5kit/examples/RL/gg colabs/logs/SAC_640000_steps.zip")
# model = model.eval()
# torch.set_grad_enabled(False)
from customModel.customModel import TorchAttentionModel3
from customModel.customModel import TorchRasterNet


# os.environ["L5KIT_DATA_FOLDER"] = "/media/pronton/linux_files/a100code/l5kit/l5kit_dataset"
# env_config_path = '/home/pronton/rl/rlhf-car/src/configs/gym_vectorizer_config.yaml'
dmg = LocalDataManager(None)
cfg = load_config_data(env_config_path)
# rollout_sim_cfg = SimulationConfigGym()
# rollout_sim_cfg.num_simulation_steps = None
# env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': rollout_sim_cfg,  'train': False, 'return_info': True, 'rescale_action': False}
# rollout_env = L5Env2(**env_kwargs)
# print(rollout_env.action_space)
# model = TorchAttentionModel3(np.zeros((112,112,7)), np.array((3,)),3, model_config= {"custom_model_config": {'cfg':cfg}}, name='')
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())
###################### TRAINING ######################
ModelCatalog.register_custom_model( "TorchSeparatedAttentionModel", TorchAttentionModel3)
ModelCatalog.register_custom_model( "TorchSeparatedRasterModel", TorchRasterNet)
from src.customEnv.wrapper import L5EnvRasterizerTorch
from ray import tune
import ray
train_eps_length = 32
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': train_sim_cfg, 'train': False}#, 'rescale_action': False}
tune.register_env("L5-CLE-V2", lambda config: L5Env2(**env_kwargs))
tune.register_env("L5-CLE-V1", lambda config: L5EnvRasterizerTorch(env = L5Env(**env_kwargs), \
                                                           raster_size= cfg['raster_params']['raster_size'][0], \
                                                           n_channels = 5))
non_kin_rescale =  L5Env(**env_kwargs)

print('non_kin_rescale:', non_kin_rescale)


ray.init(num_cpus=4, ignore_reinit_error=True, log_to_driver=False, local_mode=True)
algo = ppo.PPO(
        env="L5-CLE-V1", #"L5-CLE-V2",
        config={
            # 'disable_env_checking':True,
            "framework": "torch",
            'log_level': 'INFO',
            'num_gpu': 0,
            'train_batch_size': 1,
            'sgd_minibatch_size': 1,
            'num_sgd_iter': 1,
            'seed': 42,
            'batch_mode': 'truncate_episodes',
            # "rollout_fragment_length": 32,
            "model": {
                "custom_model": "TorchSeparatedRasterModel", #TorchSeparatedAttentionModel
                # Extra kwargs to be passed to your model's c'tor.
                "custom_model_config": {
                    'future_num_frames':cfg["model_params"]["future_num_frames"],
                    'freeze_actor': True,
                    'non_kin_rescale': non_kin_rescale,
                    },
            },            
            # "output": "/home/pronton/rl/l5kit/examples/RL/notebooks/logs/l5env2-out", 
            # "output_max_file_size": 5000000,
            '_disable_preprocessor_api': True,
        },
    )

KLPPO = 'KLPPO'
UNROLLVECTORNET = 'UNROLLVECTORNET'
VECTORNET = 'VECTORNET'
def rollout_episode(model, env, idx = 0, model_type = VECTORNET):
    """Rollout a particular scene index and return the simulation output.

    :param model: the RL policy
    :param env: the gym environment
    :param idx: the scene index to be rolled out
    :return: the episode output of the rolled out scene
    """

    # Set the reset_scene_id to 'idx'
    env.set_reset_id(idx)
    
    # Rollout step-by-step
    obs = env.reset()
    done = False
    rewards = []
    while True:
        # print(obs)
        # action = np.array(env.action_space.sample())
        if model_type == 'KLPPO':
            action = model.compute_single_action(obs)
        if model_type in ['VECTORNET', 'UNROLLNET']:
            action = model(obs)
        # print(action)
        obs, reward, done, info = env.step(action)
        rewards.append(reward)
        # print(action)
        # print(env.ego_output_dict)
        if done:
            break

    # The episode outputs are present in the key "sim_outs"
    sim_out = info["sim_outs"][0]
    return sim_out, np.array(rewards)

/workspace/source/l5kit2/l5kit/l5kit/data/zarr_dataset.py:213: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  dataset = ChunkedDataset("")
/workspace/source/l5kit2/l5kit/l5kit/simulation/utils.py:107: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  new_dataset = ChunkedDataset("")


non_kin_rescale: <L5Env instance>


2023-04-10 14:22:51,464	INFO worker.py:1538 -- Started a local Ray instance.
2023-04-10 14:22:52,825	WARNING algorithm_config.py:488 -- Cannot create PPOConfig from given `config_dict`! Property num_gpu not supported.
2023-04-10 14:22:52,826	WARNING deprecation.py:47 -- DeprecationWarning: `algo = Algorithm(env='L5-CLE-V1', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('L5-CLE-V1').build()` instead. This will raise an error in the future!
2023-04-10 14:22:52,832	WARNING unified.py:54 -- Could not instantiate JsonLogger: cannot pickle 'google.protobuf.pyext._message.RepeatedCompositeContainer' object.


TypeError: Could not serialize the argument <function WorkerSet._make_worker.<locals>.session_creator at 0x7fb14d878ca0> for a task or actor ray.rllib.evaluation.rollout_worker.RolloutWorker.__init__. Check https://docs.ray.io/en/master/ray-core/objects/serialization.html#troubleshooting for more information.

In [ ]:
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None

env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': rollout_sim_cfg,  'train': False, 'return_info': True}#, 'rescale_action': True}
# tune.register_env("L5-CLE-V0", lambda config: L5Env(**env_kwargs))
rollout_env = L5EnvRasterizerTorch(env = L5Env(**env_kwargs), \
                                                           raster_size= cfg['raster_params']['raster_size'][0], \
                                                           n_channels = 5)
# rollout_env = L5Env2(**env_kwargs)

In [ ]:
print(non_kin_rescale)

In [ ]:


rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None

env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': rollout_sim_cfg,  'train': False, 'return_info': True, 'rescale_action': False}
# rollout_env = L5Env2(**env_kwargs)
rollout_env = L5Env2(**env_kwargs)



In [ ]:
from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle, simulation_out_to_visualizer_scene
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
sim_outs =[]
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None
# env_config_path = '/workspace/source/src/configs/gym_config.yaml'

# env_config_path = 
# env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': rollout_sim_cfg,  'train': False, 'return_info': True}

# rollout_env = L5Env(**env_kwargs)

# for i in range(1):
# sim_outs.append(rollout_episode(algo, rollout_env, 82, KLPPO))
# results = [rollout_episode(algo, rollout_env, i, KLPPO) for i in [0,20,40]]
results = np.array([rollout_episode(algo, rollout_env, i, KLPPO) for i in [0]])
sim_outs,rewards = results[:,0], np.array(results[:,1])
rewards.reshape(rewards.shape[0], -1)
print([np.sum(r) for r in rewards])
print(rewards)

# might change with different rasterizer
mapAPI = MapAPI.from_cfg(dmg, cfg)

def visualize_outputs(sim_outs, map_API):
    for sim_out in sim_outs: # for each scene
        vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, map_API)
        show(visualize(sim_out.scene_id, vis_in))

output_notebook()
# visualize_outputs(sim_outs, mapAPI)

In [ ]:
train_eps_length = 10
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1

env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': train_sim_cfg}
tune.register_env("L5-CLE-V2", lambda config: L5Env2(**env_kwargs))
ray.init(num_cpus=1, ignore_reinit_error=True, log_to_driver=False)
ModelCatalog.register_custom_model( "TorchSeparatedAttentionModel", TorchAttentionModel)
ModelCatalog.register_custom_model( "TorchSharedAttentionModel", TorchAttentionModel2)
ModelCatalog.register_custom_model("GN_CNN_torch_model", TorchGNCNN)

# Create the Trainer.


## L5env1

In [ ]:
train_eps_length = 32
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': train_sim_cfg}


tune.register_env("L5-CLE-V1", lambda config: L5EnvWrapper(L5Env(**env_kwargs)))
ray.init(num_cpus=1, ignore_reinit_error=True, log_to_driver=False)
ModelCatalog.register_custom_model(
        "tcnn", TorchGNCNN
    )
# Create the Trainer.
algo = ppo.PPO(
        env="L5-CLE-V1",
        config={
            'num_worker': 1,
            'disable_env_checking':True,
            "framework": "torch",
            # "model": {
            #     "custom_model": "GN_CNN_torch_model",
            #     "custom_model_config": {'feature_dim':128},
            # },
            # "output": "/home/pronton/rl/l5kit/examples/RL/notebooks/logs/l5env2-out", 
            # "output_max_file_size": 5000000,
            '_disable_preprocessor_api': True,
        },
    )

for i in range(2):
    result = algo.train()
    print(pretty_print(result))

In [ ]:

train_eps_length = 10
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1

env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
# env = L5Env2(**env_kwargs)
# env1_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
# env1 = L5Env(**env_kwargs)


## L5env2

In [ ]:
algo = ppo.PPO(
        env="L5-CLE-V2",
        config={
            'disable_env_checking':True,
            "framework": "torch",
            'log_level': 'INFO',
            'num_gpu': 1,
            'train_batch_size': 1,
            'sgd_minibatch_size': 1,
            'num_sgd_iter': 1,
            'seed': 42,
            'batch_mode': 'truncate_episodes',
            # "rollout_fragment_length": 32,
            "model": {
                "custom_model": "TorchSharedAttentionModel",
                # Extra kwargs to be passed to your model's c'tor.
                "custom_model_config": {'cfg':cfg},
            },
            # "output": "/home/pronton/rl/l5kit/examples/RL/notebooks/logs/l5env2-out", 
            # "output_max_file_size": 5000000,
            '_disable_preprocessor_api': True,
        },
    )

for i in range(1):
    result = algo.train()
    print(pretty_print(result))